<a href="https://colab.research.google.com/github/meghasyam/python/blob/master/closuredecorator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
def outer_function():
    a = 5  # local to outer_function 
    def inner_function():
        nonlocal a
        a = 10
        print("Inner function: ",a)
    inner_function()
    print("Outer function: ",a)

outer_function()

Inner function:  10
Outer function:  10


In [0]:
def print_msg(msg):
# This is the outer enclosing function
    def printer():
# This is the nested function
        nonlocal msg
        msg = "new"
        print(msg)
    printer()

# We execute the function
# Output: Hello
print_msg("Hello")

new


In [0]:
def print_msg(msg):
# This is the outer enclosing function

    def printer():
# This is the nested function
        print(msg)

    return printer  # this got changed

# Now let's try calling this function.
# Output: Hello
another = print_msg("Hello")
another()

Hello


In [0]:
def make_multiplier_of(n):
    def multiplier(x):
        return x * n
    return multiplier  #inner functions returns

# Multiplier of 3
times3 = make_multiplier_of(3)

# Multiplier of 5
times5 = make_multiplier_of(5)

# Output: 27
print(times3(9))

# Output: 15
print(times5(3))

# Output: 30
print(times5(times3(2)))

27
15
30


In [0]:
make_multiplier_of.__closure__

In [0]:
times3.__closure__

(<cell at 0x7f0fbd94a0d8: int object at 0xa68b00>,)

In [0]:
times3.__closure__[0].cell_contents  #oops encapsule - package (data + methoeds)  - data hiding -private __ or _

3

In [0]:
times5.__closure__[0].cell_contents

5

In [0]:
def inc(x):
    return x + 1

def dec(x):
    return x - 1

def operate(func, x):
    result = func(x)
    return result

operate(inc,3)
#operate(dec,3)

4

In [0]:
def is_called():
    def is_returned():
        print("Hello")
    return is_returned

new = is_called()

#Outputs "Hello"
new()

Hello


In [0]:
def make_pretty(func):
    def inner():
        print("I got decorated")
        func()
    return inner

# ordinary -- make_pretty  -oridnary    
@make_pretty
def ordinary():
    print("I am ordinary")

@make_pretty
def ordinary1():
    print("I am ordinary-1")

@make_pretty
def ordinary2():
    print("I am ordinary-2")    

decor = ordinary
decor()
ordinary1()
ordinary2()

I got decorated
I am ordinary
I got decorated
I am ordinary-1
I got decorated
I am ordinary-2


In [0]:
print(decor.__closure__[0].cell_contents)
print(make_pretty.__closure__)

<function ordinary at 0x7f0fbd0ee378>
None


In [2]:
def smart_divide(func):  # non local 
   def inner(a,b):
      print("I am going to divide",a,"and",b)
      if b == 0:
         print("Whoops! cannot divide")
         return

      return func(a,b)
   return inner

@smart_divide
def divide(a,b):
    return a/b
divide(2,3)  # smart - divide (divide)
divide(2,0)    

I am going to divide 2 and 3
I am going to divide 2 and 0
Whoops! cannot divide


In [12]:
def works_for_all(func):
    def inner(*args, **kwargs):
        print("I can decorate any function")
        return func(*args, **kwargs)
    return inner

@works_for_all
def multi(*args,**kwargs):
    print(args) 
    print(kwargs) 


multi("new","new1","new2","new3",key="23",key1="nnn")    

I can decorate any function
('new', 'new1', 'new2', 'new3')
{'key': '23', 'key1': 'nnn'}


In [13]:
def star(func):
    def inner(*args, **kwargs):
        print("*" * 30)
        func(*args, **kwargs)
        print("*" * 30)
    return inner

def percent(func):
    def inner(*args, **kwargs):
        print("%" * 30)
        func(*args, **kwargs)
        print("%" * 30)
    return inner

@star
@percent
def printer(msg):
    print(msg)
printer("Hello")

******************************
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
Hello
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
******************************


In [17]:
def trace(f):
...     "Decorate a function to print a message before and after execution."
...     def traced(*args, **kw):
...         "Print message before and after a function call."
...         print("Entering", f.__name__)
...         result = f(*args, **kw)
...         print("Leaving", f.__name__)
...         return result
...     return traced

@trace
... def myfunc(x, a=None):
...     "Simply prints a message and arguments."
...     print("Inside myfunc")
...     print("x:", x, "a:", a)

myfunc("ONE")

Entering myfunc
Inside myfunc
x: ONE a: None
Leaving myfunc


In [21]:
trace.__name__
myfunc.__name__
myfunc.__doc__

'Print message before and after a function call.'

In [23]:
from functools import wraps
>>> def simpledec(f):
...     "A really simple decorator to demonstrate functools.wraps."
...     @wraps(f)
...     def wrapper(arg):
...         print("Calling f with arg", arg)
...         return f(arg)
...     return wrapper
... 
>>> @simpledec
... def f(x):
...     "Simply prints its argument."
...     print("Inside f, arg is", x)
... 
>>> f("Hello")
>>> f.__name__
>>> f.__doc__
>>> 

Calling f with arg Hello
Inside f, arg is Hello


'Simply prints its argument.'